In [200]:
import pandas as pd
import numpy as np
import sqlite3
import re
from collections import Counter

conn = sqlite3.connect('anime.db')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_row', 200)

df = pd.read_csv('anime.csv', index_col='MAL_ID')
my_anime = pd.read_sql('SELECT * FROM anime', conn)



In [201]:
type_dict = {
    'TV': 'ТВ',
    'Movie': 'Полнометражный фильм',
    'OVA': 'OVA',
    'Special': 'ТВ-спэшл',
    'ONA': 'ONA',
    'Music': 0,
    'Unknown': 0
}
df.Type = df.Type.apply(lambda x: type_dict[x])
df2 = df[(df.Type != 0) & (df.Aired != 'Unknown') & (df.Episodes != 'Unknown')]
df2.Aired = df2.Aired.apply(lambda x: x if len(x) < 15 else x[:15])

C:\Users\PC\AppData\Local\Temp\ipykernel_22028\627347272.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.Aired = df2.Aired.apply(lambda x: x if len(x) < 15 else x[:15])


In [202]:
to_anime_name_list = [el.lower().strip() for el in df.Name.values]
my_anime_name_list = my_anime.name_en.values
to_set = set(to_anime_name_list)
my_set = set(my_anime_name_list)
name_name = my_set & to_set


In [203]:
to_list = [
    (
        name.lower(),
        int(re.findall(r'\d{4,}', date)[0]) if re.findall(r'\d{4,}', date) else 0,
        Type,
        int(Episodes)
        )\
            for name, date, Type, Episodes in zip(
                df2.Name.values,
                df2.Aired.values,
                df2.Type.values,
                df2.Episodes.values
                )
                if name not in name_name
]

In [204]:
my_list = [
    (name, int(date), Type, int(episode)) for name, date, Type, episode in zip(
                my_anime.name_en.values,
                my_anime.release.values,
                my_anime.type.values,
                my_anime.episodes_all.values
                )
                if name not in name_name
]   

In [205]:
df3 = pd.DataFrame(to_list)

In [206]:
list_dict = {}

for myName, myYear, myType, myEp in my_list:
    list_ = []
    name_list = list(df3[(df3[1] == myYear) & (df3[2] == myType) & (df3[3] == myEp)][0].values)
    if name_list:
        for word in myName.split():
            for toName in name_list:
                if word in toName:
                    list_.append(toName)
        if list_:
            n, c = Counter(list_).most_common(1)[0]
            list_dict.update({myName: n})

In [207]:
list_duble = [el[0] for el in Counter(list_dict.values()).most_common(29)]
not_duble = [(k, v) for k, v in list_dict.items() if v not in list_duble]
duble = [(k, v) for k, v in list_dict.items() if v in list_duble]

In [208]:
l = [
    ('bokutachi wa benkyou ga dekinai 2nd season', 'bokutachi wa benkyou ga dekinai'),
    ('gintama (2017)', 'gintama.'),		
    ('high school of the dead', 'highschool of the dead'),		
    ('ikoku meiro no croisee the animation', 'ikoku meiro no croisée the animation'),		
    ('kaitou joker 3nd season', 'kaitou joker 3rd season'),	
    ('kuroko no basuke 2', 'kuroko no basket 2nd season'),		
    ('uchuu senkan tiramisu', 'uchuu senkan tiramisù'),		
    ('yuru camp 2nd season', 'yuru camp△ season 2')		
]
not_duble += l


In [209]:
to_dict_encoder = {name.lower().strip() : id for name, id in zip(df.Name.values, df.index)}
my_dict_encoder = {name : id for name, id in zip(my_anime.name_en.values, my_anime.anime_id.values)}

In [225]:
result_dict = {}
for name in name_name:
    result_dict.update({my_dict_encoder[name] : to_dict_encoder[name]})

for my, to in not_duble:
    toId = to_dict_encoder[to]
    if toId not in result_dict.values():
        result_dict.update({my_dict_encoder[my] : to_dict_encoder[to]})

In [263]:
inverted_result_dict = {v: k for k, v in result_dict.items()}

In [233]:
rating_complete = pd.read_csv('rating_complete.csv')

In [284]:
anime = pd.read_csv('anime.csv', index_col='MAL_ID')
excess = list(set(anime.index) - set(result_dict.values()))
anime = anime.drop(excess, axis = 0)
anime.index = [inverted_result_dict[i] for i in anime.index]
anime.to_csv('anime.csv')

In [285]:
anime = pd.read_csv('anime_with_synopsis.csv', index_col='MAL_ID')
excess = list(set(anime.index) - set(result_dict.values()))
anime = anime.drop(excess, axis = 0)
anime.index = [inverted_result_dict[i] for i in anime.index]
anime.to_csv('anime_with_synopsis.csv')

In [ ]:
anime = pd.read_csv('animelist.csv', index_col='anime_id')

In [288]:
excess = list(set(anime.index) - set(result_dict.values()))
anime = anime.drop(excess, axis = 0)
anime.index = [inverted_result_dict[i] for i in anime.index]

In [289]:
anime

,user_id,rating,watching_status,watched_episodes
1318,0,9,1,1
649,0,7,1,4
1797,0,0,1,1
2710,0,10,1,0
585,0,9,1,5
...,...,...,...,...
1507,353404,10,1,77
1504,353404,9,2,153
1506,353404,8,2,64
1505,353404,9,2,291


In [290]:
anime.info

<bound method DataFrame.info of       user_id  rating  watching_status  watched_episodes
1318        0       9                1                 1
649         0       7                1                 4
1797        0       0                1                 1
2710        0      10                1                 0
585         0       9                1                 5
...       ...     ...              ...               ...
1507   353404      10                1                77
1504   353404       9                2               153
1506   353404       8                2                64
1505   353404       9                2               291
356    353404       9                2                24

[66671514 rows x 4 columns]>

In [291]:
anime.to_csv('animelist.csv')
anime = pd.read_csv('rating_complete.csv', index_col='anime_id')
anime.info

<bound method DataFrame.info of           user_id  rating
anime_id                 
430             0       9
1004            0       5
3010            0       7
570             0       7
2762            0       9
...           ...     ...
502        353404       8
987        353404       4
225        353404       8
243        353404       7
896        353404       7

[57633278 rows x 2 columns]>

In [292]:
excess = list(set(anime.index) - set(result_dict.values()))
anime = anime.drop(excess, axis = 0)
anime.index = [inverted_result_dict[i] for i in anime.index]
anime.info
anime.to_csv('rating_complete.csv')

2152